# Image processing language


In [1]:
import rawpy
import imageio
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob

def black_level_correction(image, BL, clip=True):
    try:
        iter(BL)
    except TypeError:
        BL = np.full(3, BL)
    
    result = (image - np.expand_dims(BL, axis=[0,1]))
    if clip:
        return np.clip(result, 0., 1.)
    else:
        return result

def crop_and_center(filename, width=1000):
    raw = rawpy.imread(filename).raw_image
    colour = cv2.cvtColor(raw, cv2.COLOR_BAYER_BG2BGR)
    del raw
    processed = black_level_correction(colour, np.percentile(colour, axis=(0,1), q=1e-6), clip=False)
    processed /= np.expand_dims(np.percentile(processed, q=100. - 1e-5, axis=[0,1]), axis=[0,1])

    center_x = int(np.average(np.arange(processed.shape[0]), weights=np.sum(np.heaviside(processed[:, :, 1] - processed[:, :, 1].max()/10., 0.), axis=1)))
    center_y = int(np.average(np.arange(processed.shape[1]), weights=np.sum(np.heaviside(processed[:, :, 1] - processed[:, :, 1].max()/10., 0.), axis=0)))

    processed = processed[center_x-width:center_x+width, center_y-width:center_y+width]

    return processed

In [2]:
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from functools import partial
image_names = sorted(glob("./*.CR2"))

for i in np.arange(4357, len(image_names)):
    try:
        print("Process image %s" % image_names[i] + "\t%i/%i   [%.2f" % (i+1, len(image_names), 100. * (i+1) / len(image_names)) + r"%]", end="\r")
        fig, ax = plt.subplots(figsize=(10, 10), dpi=200)
        img = ax.imshow(np.clip(crop_and_center(image_names[i]), 0., 1.))
        ax.axis("off")
        ax.set(position=[0,0,1,1])
        fig.savefig('converted/frame_%s.jpg' % str(i).zfill(5)) 
        plt.close("all")
    except:
        plt.close("all")
        print("i = %i" % i, "Could not generate cropped image for file %s. We are skipping this one" % image_names[i])

In [3]:
import cv2
from tqdm import tqdm
video_name = 'converted/converted.mp4'

images = sorted(glob("converted/frame*.jpg"))
frame = cv2.imread(images[0])
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"mp4v"), 30, (width,height)) # "DIVX", "XVID", "MJPG", "X264", "WMV1", "WMV2", "FMP4", "mp4v", "avc1", "I420", "IYUV", "mpg1", "H264"

for image in tqdm(images):
    video.write(cv2.rotate(cv2.imread(image), cv2.ROTATE_180))

cv2.destroyAllWindows()
video.release()

100%|██████████| 4591/4591 [05:05<00:00, 15.01it/s]


# Test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rawpy
from rawkit.raw import Raw
from matplotlib.patches import Circle
import PIL.Image
from tqdm import tqdm

def convert_raw_image(fname):
    """Display an rgb raw image."""
    raw_data = rawpy.imread(fname)
    rgb = raw_data.postprocess(gamma=(1,1), no_auto_bright=True, output_bps=16) / (2**16 - 1)
    return rgb

def find_center(data, channel=1):
    """Find the image's center of mass."""
    y_center = np.average(np.arange(data.shape[1]), weights=np.sum(data[:, :, channel], axis=0))
    x_center = np.average(np.arange(data.shape[0]), weights=np.sum(data[:, :, channel], axis=1))

    return (x_center, y_center)

def regress_to_center(data, maxiter=100, init=None, stepsize=[2,2,2]): # initial_parameters [x, y, radius]

    total_integral = data.sum()
    total_area = data.size
    xx, yy = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
    
    def loss_func(x, y, r):
        mask = (xx - x)**2 + (yy - y)**2 <= r**2
        integral = np.sum(data * mask)
        area = np.pi * r**2
        
        #return area / integral - (total_area - area) / (total_integral - integral)
        return area/total_area - integral/total_integral

    if init == None:
        all_x    = [data.shape[0]      / 2.]
        all_y    = [data.shape[1]      / 2.]
        all_r    = [np.min(data.shape) / 2.]
        all_loss = [loss_func(all_x[-1], all_y[-1], all_r[-1])]
    else:
        all_x    = [init[0]]
        all_y    = [init[1]]
        all_r    = [init[2]]
        all_loss = [loss_func(all_x[-1], all_y[-1], all_r[-1])]
        
    for i in tqdm(np.arange(maxiter)):
        x = all_x[-1] + np.random.normal(0, stepsize[0])
        y = all_y[-1] + np.random.normal(0, stepsize[1])
        r = all_r[-1] + np.random.normal(0, stepsize[2])
        loss = loss_func(x, y, r)
        if all_loss[-1] >= loss:
            all_x.append(x)
            all_y.append(y)
            all_r.append(r)
            all_loss.append(loss)

    return all_x, all_y, all_r, all_loss

In [ ]:
image_data = convert_raw_image("data/IMG_1369.CR2")
center_coordinates = find_center(image_data >= 0.1)
masked_data = image_data[:, :, 1] >= 1./20.
all_x, all_y, all_r, all_loss = regress_to_center(masked_data, maxiter=200, init=[center_coordinates[1], center_coordinates[0], 482.], stepsize=[10., 10., 0.1])

print(all_x, all_y, all_r, all_loss)

In [ ]:
fig, axs = plt.subplots(nrows=4)
axs[0].plot(all_x)
axs[1].plot(all_y)
axs[2].plot(all_r)
axs[3].plot(all_loss)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.imshow(np.transpose(image_data, axes=[1,0,2]) / image_data.max())
center_coordinates = find_center(image_data >= 0.1)
ax.add_patch(Circle(
    (all_y[-1], all_x[-1]),
    radius=all_r[-1],
    fc="None",
    ec="red"
))

for x, y, r in zip(all_x, all_y, all_r):
    ax.add_patch(Circle((y, x), r, fc="None", ec="white", alpha=0.1))

ax.set_xlabel("x-coordinate")
ax.set_ylabel("y-coordinate")

ax.set_xlim(center_coordinates[0] - 750, center_coordinates[0] + 750)
ax.set_ylim(center_coordinates[1] - 750, center_coordinates[1] + 750)
plt.show()